##Import_Libraries

In [121]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import  RandomForestRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn import svm
from sklearn.neighbors import KNeighborsRegressor
from sklearn import linear_model
from sklearn.svm import SVR
from sklearn.multioutput import MultiOutputRegressor
import xgboost as xgb

from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
import sklearn.metrics

from sklearn.preprocessing import MinMaxScaler

from sklearn.decomposition import PCA
import pickle

In [122]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [123]:
!pip install mediapipe

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [124]:
import os,cv2,math,glob,random
import scipy.io as sio
from math import cos, sin
from pathlib import Path
import mediapipe
import scipy.io as sio
from math import cos, sin
from pathlib import Path
import mediapipe as mp
import warnings
warnings.filterwarnings('ignore')
from google.colab.patches import cv2_imshow

##Read_Dataset

In [125]:
data = pd.read_csv("/content/data_set.csv")
data

,Unnamed: 0,ID,F1,F2,F3,F4,F5,F6,F7,F8,...,F930,F931,F932,F933,F934,F935,F936,Pitch,Yaw,Roll
0,0,image00002.jpg,0.485651,0.686890,0.489194,0.639721,0.489802,0.647256,0.479730,0.562529,...,0.492813,0.541429,0.500371,0.665668,0.471336,0.676091,0.464005,-0.399231,0.018227,0.085676
1,1,image00004.jpg,0.441502,0.642091,0.417273,0.592251,0.448839,0.610546,0.435994,0.545069,...,0.511724,0.500658,0.514913,0.584451,0.500244,0.594060,0.495387,0.470065,1.189533,0.300959
2,2,image00006.jpg,0.317797,0.799190,0.321328,0.776498,0.318316,0.782066,0.311064,0.745973,...,0.720868,0.328639,0.723342,0.373404,0.713071,0.377643,0.709798,-0.184650,0.881137,-0.236852
3,3,image00008.jpg,0.503922,0.694309,0.469521,0.653913,0.489504,0.658751,0.444433,0.590285,...,0.510606,0.490586,0.518652,0.595615,0.455611,0.602670,0.444623,-0.175379,0.299208,-0.373374
4,4,image00013.jpg,0.510238,0.669038,0.492729,0.604797,0.499756,0.624468,0.467368,0.542362,...,0.488652,0.505667,0.492681,0.602484,0.452699,0.611310,0.448576,-0.026812,0.011965,-0.220662
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1848,1848,image04345.jpg,0.515443,0.683951,0.532837,0.625785,0.518815,0.639885,0.511650,0.560010,...,0.503230,0.540073,0.508401,0.629060,0.479723,0.635765,0.474628,-0.306358,-0.283822,0.038554
1849,1849,image04348.jpg,0.543907,0.689844,0.565286,0.635285,0.549289,0.646834,0.545585,0.562769,...,0.500254,0.574974,0.506460,0.659800,0.479817,0.667754,0.470887,-0.367547,-0.429723,0.122791
1850,1850,image04356.jpg,0.586373,0.680103,0.604407,0.627700,0.593793,0.645848,0.602947,0.568153,...,0.539478,0.648903,0.541426,0.734017,0.547668,0.743284,0.540139,-0.156035,0.567114,-0.108536
1851,1851,image04358.jpg,0.491617,0.692910,0.500626,0.637631,0.497282,0.649833,0.490064,0.559349,...,0.496594,0.550279,0.502602,0.673963,0.481876,0.684592,0.474287,-0.197102,-0.070430,0.105118


##Data_Preprocessing

In [126]:
datacentered = data.copy()
data.drop(['Unnamed: 0', 'ID'], axis=1, inplace=True)
num_of_keypoints = 468
num_of_features = num_of_keypoints * 2
num_of_labels = 3

def keypointscenter(df):
  for i in range(1, num_of_features + 1):
    feature = 'F' + str(i)
    if i % 2 != 0:
      df[feature] = df[feature] - data['F3']
    else:
      df[feature] = df[feature] - data['F4']
  


keypointscenter(datacentered)
datacentered.head()

,Unnamed: 0,ID,F1,F2,F3,F4,F5,F6,F7,F8,...,F930,F931,F932,F933,F934,F935,F936,Pitch,Yaw,Roll
0,0,image00002.jpg,-0.003543,0.047169,0.0,0.0,0.000608,0.007535,-0.009464,-0.077192,...,-0.146908,0.052235,-0.139350,0.176474,-0.168385,0.186897,-0.175716,-0.399231,0.018227,0.085676
1,1,image00004.jpg,0.024229,0.049840,0.0,0.0,0.031566,0.018295,0.018721,-0.047182,...,-0.080527,0.083385,-0.077338,0.167178,-0.092007,0.176787,-0.096864,0.470065,1.189533,0.300959
2,2,image00006.jpg,-0.003530,0.022692,0.0,0.0,-0.003012,0.005568,-0.010264,-0.030525,...,-0.055630,0.007312,-0.053156,0.052077,-0.063427,0.056315,-0.066700,-0.184650,0.881137,-0.236852
3,3,image00008.jpg,0.034401,0.040395,0.0,0.0,0.019983,0.004837,-0.025088,-0.063628,...,-0.143307,0.021064,-0.135261,0.126094,-0.198303,0.133149,-0.209290,-0.175379,0.299208,-0.373374
4,4,image00013.jpg,0.017510,0.064241,0.0,0.0,0.007027,0.019671,-0.025361,-0.062435,...,-0.116145,0.012938,-0.112116,0.109755,-0.152098,0.118581,-0.156221,-0.026812,0.011965,-0.220662


In [127]:
data.head()

,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,...,F930,F931,F932,F933,F934,F935,F936,Pitch,Yaw,Roll
0,0.485651,0.686890,0.489194,0.639721,0.489802,0.647256,0.479730,0.562529,0.490558,0.621213,...,0.492813,0.541429,0.500371,0.665668,0.471336,0.676091,0.464005,-0.399231,0.018227,0.085676
1,0.441502,0.642091,0.417273,0.592251,0.448839,0.610546,0.435994,0.545069,0.415757,0.576966,...,0.511724,0.500658,0.514913,0.584451,0.500244,0.594060,0.495387,0.470065,1.189533,0.300959
2,0.317797,0.799190,0.321328,0.776498,0.318316,0.782066,0.311064,0.745973,0.321459,0.767816,...,0.720868,0.328639,0.723342,0.373404,0.713071,0.377643,0.709798,-0.184650,0.881137,-0.236852
3,0.503922,0.694309,0.469521,0.653913,0.489504,0.658751,0.444433,0.590285,0.461756,0.637613,...,0.510606,0.490586,0.518652,0.595615,0.455611,0.602670,0.444623,-0.175379,0.299208,-0.373374
4,0.510238,0.669038,0.492729,0.604797,0.499756,0.624468,0.467368,0.542362,0.488200,0.584701,...,0.488652,0.505667,0.492681,0.602484,0.452699,0.611310,0.448576,-0.026812,0.011965,-0.220662


##Split data

In [95]:
x = data.iloc[:, :num_of_features]
y = data.iloc[:, -num_of_labels:]
print('x shape: ', x.shape)
print('y shape: ', y.shape)

x shape:  (1853, 936)
y shape:  (1853, 3)


In [ ]:
y

,pitch,yaw,roll
0,-1.975403,66.214500,11.572282
1,-19.185266,25.210886,-11.040552
2,-6.741508,-12.655412,17.060774
3,-0.674434,5.298100,-1.427274
4,-0.885785,-19.488333,12.369505
...,...,...,...
1848,-23.524061,-4.513399,8.109061
1849,-20.885176,5.792963,-1.419527
1850,0.536577,-6.357893,8.328095
1851,-27.264402,38.782444,-46.335663


In [15]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=123)

##Validation for data

In [128]:
def crossvalidrepeated(model, x_train, y_train):
  # define the evaluation procedure
  cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)

  # evaluate the model and collect the scores
  n_scores = cross_val_score(model, x_train, y_train, scoring='r2', cv=cv, n_jobs=-1)

  # # force the scores to be positive
  n_scores = np.absolute(n_scores)

  return np.mean(n_scores)



def crossval(x_train, y_train):
  cv_scores_dict = {}
# Tree Models
  rf_model = RandomForestRegressor(n_estimators = 3, max_features = 'sqrt', max_depth = 5, random_state = 18)
  cv_scores_dict['RandomForestRegressor'] = crossvalidrepeated(rf_model, x_train, y_train)

  # KNN Model
  knn_model = KNeighborsRegressor(5,leaf_size=15, weights='uniform')       
  cv_scores_dict['KNN_uniform'] = crossvalidrepeated(knn_model, x_train, y_train)

  # KNN Model
  knn_model = KNeighborsRegressor(5, leaf_size=15, weights='distance')       
  cv_scores_dict['KNN_distance'] = crossvalidrepeated(knn_model, x_train, y_train)

  return cv_scores_dict


crossval(x_train, y_train)

{'KNN_distance': 0.5807202501427248,
 'KNN_uniform': 0.574672886409433,
 'RandomForestRegressor': 1.0192037592921737}

In [129]:
def show(searcher, grid_param_1, grid_param_2, name_param_1, name_param_2, rot=0):
    scores_mean = searcher.cv_results_['mean_test_score']
    scores_mean = np.array(scores_mean).reshape(len(grid_param_1), len(grid_param_2)).T

    print('Best params = {}'.format(searcher.best_params_))
    print('Best score = {}'.format(scores_mean.max()))

    _, ax = plt.subplots(1,1)

    # Param1 is the X-axis, Param 2 is represented as a different curve (color line)
    for idx, val in enumerate(grid_param_2):
        ax.plot(grid_param_1, scores_mean[idx, :], '-o', label=name_param_2 + ': ' + str(val))

    ax.tick_params(axis='x', rotation=rot)
    ax.set_title('Grid Search Scores')
    ax.set_xlabel(name_param_1)
    ax.set_ylabel('CV score')
    ax.legend(loc='best')
    ax.grid('on')



def gs_knn(x_train, y_train):
  knn_model = KNeighborsRegressor()       
  weights = ['uniform', 'distance']
  n_neighbors = list(range(2,25))
  leaf_size =  [15]
  params = [{'n_neighbors': n_neighbors,
          'weights': weights,
          'leaf_size': leaf_size}]

  knn_gs = GridSearchCV(knn_model, params, scoring='r2')

  knn_gs.fit(x_train, y_train)

  show(knn_gs, n_neighbors, weights,  'n_neighbors', 'weights', 80)

In [130]:
x_train

,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,...,F927,F928,F929,F930,F931,F932,F933,F934,F935,F936
1205,0.444985,0.676184,0.415421,0.617573,0.450216,0.639141,0.436944,0.552185,0.413605,0.598053,...,0.535640,0.503110,0.523298,0.508166,0.511527,0.512501,0.611307,0.507280,0.623670,0.498255
1613,0.464962,0.650478,0.458819,0.598221,0.468952,0.617105,0.458280,0.546908,0.458800,0.581384,...,0.534699,0.508813,0.523489,0.513161,0.515171,0.515771,0.611475,0.501414,0.621746,0.494487
602,0.472881,0.672037,0.482833,0.637919,0.482050,0.642396,0.489703,0.557959,0.488375,0.620262,...,0.585331,0.500377,0.572547,0.503839,0.564278,0.509423,0.692889,0.513151,0.703791,0.508890
0,0.485651,0.686890,0.489194,0.639721,0.489802,0.647256,0.479730,0.562529,0.490558,0.621213,...,0.559712,0.485137,0.548437,0.492813,0.541429,0.500371,0.665668,0.471336,0.676091,0.464005
535,0.496166,0.697579,0.495093,0.637115,0.500488,0.648536,0.487008,0.557195,0.495945,0.617223,...,0.581397,0.482583,0.566829,0.490175,0.557806,0.497575,0.691541,0.473700,0.701776,0.468630
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1122,0.450817,0.659192,0.422829,0.610530,0.457376,0.629029,0.441971,0.566108,0.421074,0.595887,...,0.540171,0.530566,0.527251,0.534807,0.514775,0.537712,0.609321,0.528829,0.621766,0.519163
1346,0.502469,0.678505,0.530895,0.621958,0.514156,0.637101,0.527069,0.550146,0.537783,0.602370,...,0.577311,0.510516,0.568911,0.512671,0.566281,0.515199,0.659346,0.520230,0.668017,0.516821
1406,0.535756,0.674886,0.549139,0.618261,0.532575,0.635270,0.512314,0.559533,0.548158,0.600052,...,0.541171,0.491597,0.534570,0.498843,0.533406,0.503917,0.619016,0.463167,0.624797,0.449948
1389,0.473812,0.663364,0.441244,0.620635,0.471580,0.630647,0.447504,0.577602,0.437950,0.607768,...,0.509862,0.524873,0.502036,0.531041,0.494018,0.535866,0.556977,0.504825,0.565902,0.496546


In [131]:
y_train

,Pitch,Yaw,Roll
1205,-0.045438,1.197566,0.031494
1613,0.075674,0.387528,0.034612
602,-0.554661,0.164135,0.269781
0,-0.399231,0.018227,0.085676
535,-0.214652,0.237559,0.041425
...,...,...,...
1122,-0.180190,1.237419,-0.151969
1346,-0.236607,-0.200545,0.303587
1406,-0.331991,-0.280685,-0.057327
1389,-0.492415,0.955447,-0.468950


##Train_Different_Models

In [27]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split
neigh = KNeighborsRegressor(n_neighbors=10,leaf_size=15)
# # X=df.drop(['yaw','pitch','roll','img'],axis=1)
# #Y=y[['yaw']]
# # X_train, X_test, y_train, y_test = train_test_split(x, Y,test_size=0.05, random_state=42)
neigh.fit(x_train, y_train['Yaw']).score(x_test, y_test['Yaw'])
# def apply_knn(x_train, y_train, x_test, y_test, n_neighbors):
#   knn_model = KNeighborsRegressor(n_neighbors=n_neighbors, leaf_size=15).fit(x_train, y_train)
#   score_knn = knn_model.score(x_test, y_test)
#   print(score_knn)
#   return knn_model

0.7096067241875277

In [ ]:
# apply_knn(x_train, y_train['yaw'], x_test, y_test['yaw'], 9)

In [38]:
neigh = KNeighborsRegressor(n_neighbors=80,leaf_size=15)
# # X=df.drop(['yaw','pitch','roll','img'],axis=1)
# #Y=y[['yaw']]
# # X_train, X_test, y_train, y_test = train_test_split(x, Y,test_size=0.05, random_state=42)
neigh.fit(x_train, y_train['Pitch']).score(x_test, y_test['Pitch'])

0.020106034983369825

In [42]:
neigh = KNeighborsRegressor(n_neighbors=4)
# # X=df.drop(['yaw','pitch','roll','img'],axis=1)
# #Y=y[['yaw']]
# # X_train, X_test, y_train, y_test = train_test_split(x, Y,test_size=0.05, random_state=42)
neigh.fit(x_train, y_train['Roll']).score(x_test, y_test['Roll'])

0.5489306198942783

In [44]:
def apply_knn(x_train, y_train, x_test, y_test, n_neighbors):
  knn_model = KNeighborsRegressor(n_neighbors=n_neighbors, leaf_size=15).fit(x_train, y_train)
  score_knn = knn_model.score(x_test, y_test)
  print(score_knn)
  return knn_model

In [45]:
multi_label_knn = apply_knn(x_train, y_train, x_test, y_test, 17)

0.2955472212329663


In [ ]:
with open('multi_label_knn_3.pkl', 'wb') as file:
        pickle.dump(multi_label_knn, file)


In [46]:
def apply_random_forest(x_train, y_train, x_test, y_test):
  rf = RandomForestRegressor(bootstrap= True, max_depth= None, max_features= 'sqrt', min_samples_leaf= 4, min_samples_split= 5, n_estimators= 5)
  rf.fit(x_train, y_train)
  
  print(rf.score(x_test, y_test))
  return rf

In [47]:
apply_random_forest(x_train, y_train['Yaw'],  x_test, y_test['Yaw'])

0.7001228851989683


RandomForestRegressor(max_features='sqrt', min_samples_leaf=4,
                      min_samples_split=5, n_estimators=5)

In [48]:
apply_random_forest(x_train, y_train['Pitch'],  x_test, y_test['Pitch'])


-0.4231620875129518


RandomForestRegressor(max_features='sqrt', min_samples_leaf=4,
                      min_samples_split=5, n_estimators=5)

In [49]:
apply_random_forest(x_train, y_train['Roll'],  x_test, y_test['Roll'])

0.3306550055814774


RandomForestRegressor(max_features='sqrt', min_samples_leaf=4,
                      min_samples_split=5, n_estimators=5)

In [50]:
multi_label_rf = apply_random_forest(x_train, y_train,  x_test, y_test)

0.12470537355530127


In [51]:
def apply_svr(x_train, y_train, x_test, y_test, gamma=0.1, c=1):
  if len(y_train.shape) > 1:
    model = MultiOutputRegressor(SVR(gamma= gamma, C=c)).fit(x_train, y_train)
  else:
    model = SVR(gamma= gamma).fit(x_train, y_train)
 
  train_score_svr = model.score(x_train, y_train)
  test_score_svr = model.score(x_test, y_test)
  print('train_score_svr:  ', train_score_svr)
  print('test_score_svr:  ', test_score_svr)

  return model

In [53]:
svr_yaw = apply_svr(x_train, y_train['Yaw'], x_test, y_test['Yaw'], 0.1, 0.4)

train_score_svr:   0.9477124206388524
test_score_svr:   0.6499752730450199


In [62]:
svr_pitch = apply_svr(x_train, y_train['Pitch'], x_test, y_test['Pitch'], 0.009, 0.4)

train_score_svr:   0.16962126739470706
test_score_svr:   0.12206298342634614


In [65]:
svr_roll = apply_svr(x_train, y_train['Roll'], x_test, y_test['Roll'], 0.009, 0.4)

train_score_svr:   0.3177598912249705
test_score_svr:   0.16003611095514514


In [68]:
multi_label_svr = apply_svr(x_train, y_train, x_test, y_test, 0.009, 1)

train_score_svr:   0.4560613358767114
test_score_svr:   0.3375704461274953


In [69]:
with open('multi_label_svr6.pkl', 'wb') as file:
        pickle.dump(multi_label_svr, file)

In [70]:
def apply_xgb(x_train, y_train, x_test, y_test, max_depth = 5, alpha = 10, n_estimators = 10):
  if len(y_train.shape) > 1:
    model = MultiOutputRegressor(xgb.XGBRegressor(max_depth = max_depth, alpha = alpha, n_estimators = n_estimators)).fit(x_train, y_train)
              
  else:
    model = xgb.XGBRegressor( max_depth = max_depth, alpha = alpha, n_estimators = n_estimators).fit(x_train, y_train)
  train_score = model.score(x_train, y_train)
  test_score = model.score(x_test, y_test)

  print('train_score_svr:  ', train_score)
  print('test_score_svr:  ', test_score)

  return model

In [72]:
xg_reg_pitch = apply_xgb(x_train, y_train['Pitch'], x_test, y_test['Pitch'], n_estimators=9, max_depth = 9)

[18:44:57] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
train_score_svr:   0.4485222441499005
test_score_svr:   -1.337414878177162


In [73]:
xg_reg_yaw = apply_xgb(x_train, y_train['Yaw'], x_test, y_test['Yaw'])

[18:45:06] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
train_score_svr:   0.7496989079536862
test_score_svr:   0.5474200315970086


In [74]:
xg_reg_roll = apply_xgb(x_train, y_train['Roll'], x_test, y_test['Roll'], n_estimators=16, max_depth = 13)

[18:45:13] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
train_score_svr:   0.8124544614457702
test_score_svr:   0.5705504984487666


In [75]:
multi_label_xg_reg = apply_xgb(x_train, y_train, x_test, y_test, n_estimators=20, max_depth = 5)

[18:45:22] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:45:27] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:45:31] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
train_score_svr:   0.8844489776004206
test_score_svr:   -0.42335929533668826


##Test_Model

In [114]:
def center_of_feature(feature):
  for i in range(len(feature)):
    copy = feature.copy()
    if i % 2 == 0:
      feature[i] = feature[i] - copy[2]
    
    else:
      feature[i] = feature[i] - copy[3]

In [115]:
def drawaxis(img, pitch,yaw,roll, tdx=None, tdy=None, size = 100):

    yaw = -yaw
    if tdx != None and tdy != None:
        tdx = tdx
        tdy = tdy
    else:
        height, width = img.shape[:2]
        tdx = width / 2
        tdy = height / 2

    # X-Axis pointing to right. drawn in red
    x1 = size * (cos(yaw) * cos(roll)) + tdx
    y1 = size * (cos(pitch) * sin(roll) + cos(roll) * sin(pitch) * sin(yaw)) + tdy

    # Y-Axis | drawn in green
    #        v
    x2 = size * (-cos(yaw) * sin(roll)) + tdx
    y2 = size * (cos(pitch) * cos(roll) - sin(pitch) * sin(yaw) * sin(roll)) + tdy

    # Z-Axis (out of the screen) drawn in blue
    x3 = size * (sin(yaw)) + tdx
    y3 = size * (-cos(yaw) * sin(pitch)) + tdy

    cv2.line(img, (int(tdx), int(tdy)), (int(x1),int(y1)),(0,0,255),3)
    cv2.line(img, (int(tdx), int(tdy)), (int(x2),int(y2)),(0,255,0),3)
    cv2.line(img, (int(tdx), int(tdy)), (int(x3),int(y3)),(255,0,0),2)

    return img

In [116]:
pkl_filename = '/content/multi_label_svr6.pkl'
with open(pkl_filename, 'rb') as file:
  pretrained_model = pickle.load(file)


In [117]:
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(min_detection_confidence=0.5, min_tracking_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils
drawing_spec = mp_drawing.DrawingSpec(thickness=1, circle_radius=1)
mp_drawing_styles = mp.solutions.drawing_styles

In [118]:
import matplotlib.pyplot as plt
cap = cv2.VideoCapture('/content/drive/MyDrive/Pose_project/video222.mp4')
i = 0
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))


fourcc = cv2.VideoWriter_fourcc('M','J','P','G')   
out = cv2.VideoWriter('final_trial_svr.avi', fourcc, fps, (frame_width, frame_height))
 
while cap.isOpened():
    ret, image = cap.read()

    if not ret:
      break

    # Flip the image horizontally for a later selfie-view display
    # Also convert the color space from BGR to RGB
    image = cv2.cvtColor(cv2.flip(image, 1), cv2.COLOR_BGR2RGB)

    # To improve performance
    image.flags.writeable = False
    
    # Get the result
    results = face_mesh.process(image)
    
    # To improve performance
    image.flags.writeable = True
    
    # Convert the color space from RGB to BGR
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
 
    # for landmark in face.landmark:
    img_h, img_w, img_c = image.shape
    face_3d = []
    face_2d = []
    if results.multi_face_landmarks != None:
        for face_landmarks in results.multi_face_landmarks:
            featuers = []
            for idx, lm in enumerate(face_landmarks.landmark):
                if idx == 1:
                    nose_2d = (lm.x * img_w, lm.y * img_h)
                    nose_3d = (lm.x * img_w, lm.y * img_h, lm.z * 3000)

                featuers.append(lm.x)
                featuers.append(lm.y)

         
            nose_pt = (int(nose_2d[0]), int(nose_2d[1]))
            featuers = np.array(featuers).reshape(1,-1)
            
            featuers_df = pd.DataFrame(featuers, columns=x_train.columns)

         
            y_pred = pretrained_model.predict(featuers_df)

            pitch = y_pred[0][0]
            yaw = y_pred[0][1]
            roll = y_pred[0][2]

        


            image = drawaxis(image,pitch,yaw,roll,nose_pt[0],nose_pt[1])


        
    out.write(image)

    if cv2.waitKey(20) == ord('q'):
        break

    i+=1

cv2.destroyAllWindows()
cap.release()
# print('done')       